In [ ]:


from time import time

# Benchmark tijd
global benchmark

# Reset de tijd van de benchmark
def benchmark_reset():
    global benchmark
    benchmark = time()
    
# Verkrijg de 'tijd' van de benchmark 
def benchmark_get():
    global benchmark
    return str(round(time() - benchmark, 2)) + "s"

# Functie om een csv in te kunnen lezen, de header te verwijderen en elke regel te splitten (met "::" erin)
# @param = csv bestand; @return = csv bestand zonder header en gesplit;
def csvreader(csv_file):
    print("Loading file", csv_file, "...")
    reader = sc.textFile(csv_file)
    header = reader.first()
    filterHeader = reader.filter(lambda x: x != header)
    splitRow = filterHeader.map(lambda x: x.split('::'))
    return splitRow

# Lees ratings_train bestand en sla het op in een variabele
benchmark_reset()
csvRatingTrain = csvreader("./ratings_train.csv")

# Lees ratings_test bestand en sla het op in een variabele
csvRatingTest = csvreader("./ratings_test.csv")
print("Het laden van de bestanden duurde: ", benchmark_get(), "\n")

benchmark_reset()
print("Aan het sorteren ...")

# Haal alle geruikers eruit
ratingZonderGebruikers = csvRatingTrain.map(lambda x: x[1:3])

# ......................... en sorteer ze
gesorteerdeData = ratingZonderGebruikers.map(lambda x: (x[0], float(x[1]))).groupByKey()

# Filter alleen de films met 10.000 0f meer ratings
filmsMetHogeScore = gesorteerdeData.filter(lambda x: len(x[1]) >= 20000)

# 
aantalFilmRatings = filmsMetHogeScore.map(lambda x: (x[0], x[1], len(x[1])))

#
gemiddelde = aantalFilmRatings.map(lambda x: (x[0], sum(x[1]) / len(x[1]), x[2]))

# De films op volgorde van ratings (hoog - laag)
films = gemiddelde.sortBy(lambda x: x[2], ascending=False)

# Laat tabel zien met gegevens
films.toDF(['Film ID', 'Beoordeling', 'Aantal ratings']).show()

print("Aantal films in de dataset:", films.count())

print("Het sorteren en laten zien van de tabel duurde:", benchmark_get(), "\n")

Loading file ./ratings_train.csv ...
Loading file ./ratings_test.csv ...
Het laden van de bestanden duurde:  3.82s 

Aan het sorteren ...
+-------+------------------+--------------+
|Film ID|       Beoordeling|Aantal ratings|
+-------+------------------+--------------+
|    296| 4.171089877038746|         66769|
|    356| 4.024502624191383|         65544|
|    593| 4.174403785187427|         62771|
|    318| 4.445266083178943|         62612|
|    480|3.6606881097201396|         59351|
|    260| 4.186831321503402|         53931|
|    110| 4.036158521668768|         53141|
|    589|3.9282337732499806|         51828|
|   2571| 4.184518080571978|         50911|
|    527| 4.306556713548518|         49415|
|    457|3.9827953931465947|         49231|
|      1| 3.914494583367272|         49108|
|    150| 3.865528787432085|         47486|
|    780|3.3601515704502027|         46579|
|   1210| 4.001419324315606|         46501|
|     50| 4.330394044965702|         46213|
|    592|3.399701642057580

In [3]:
benchmark_reset()


gebruikerFilms = csvRatingTrain.map(lambda x: x[0:2]).groupByKey().mapValues(list)

# Maak een lijst met de films die de gebruiker gezien heeft
listGebruikerFilms = gebruikerFilms.collect()

# Maak een lijst van de beste films
listBesteFilms = films.map(lambda x: x[0]).collect()

# Laat tabel zien met gegevens
gebruikerFilms.toDF(['Gebruiker ID', 'Film ID']).show()

print("Het laden van de bekeken films van de gebruikers duurde:", benchmark_get(), "\n")

+------------+--------------------+
|Gebruiker ID|             Film ID|
+------------+--------------------+
|       62505|[1, 17, 32, 47, 1...|
|       67056|[47, 50, 147, 175...|
|       28916|[2, 18, 19, 23, 2...|
|       75265|[110, 188, 366, 3...|
|        7505|[1, 4, 5, 7, 11, ...|
|       19307|[16, 173, 246, 43...|
|       20711|[34, 110, 111, 16...|
|        5964|[3, 70, 110, 240,...|
|       42659|[110, 260, 329, 3...|
|       66560|[1, 10, 16, 19, 2...|
|       61988|[10, 296, 317, 50...|
|       50581|[19, 31, 256, 327...|
|      130731|[1, 4, 6, 7, 10, ...|
|       58505|[1, 34, 47, 110, ...|
|       29109|[410, 466, 468, 7...|
|       25868|[3, 5, 7, 11, 17,...|
|       60529|[2, 10, 20, 104, ...|
|       71491|[593, 788, 1517, ...|
|       42508|[1, 2, 3, 5, 6, 7...|
|       13896|[6, 10, 32, 70, 2...|
+------------+--------------------+
only showing top 20 rows

Het laden van de bekeken films van de gebruikers duurde: 119.75s 



In [5]:
benchmark_reset()

lijst_nieuw = []

listBesteFilms_set = set(listBesteFilms)
for user, user_films in listGebruikerFilms:
    for not_user_film in listBesteFilms_set-set(user_films):
        lijst_nieuw.append([user, not_user_film])
    
print("Het aanbevelen van de films duurde:", benchmark_get(), "\n")

Het aanbevelen van de films duurde: 23.83s 



In [6]:
benchmark_reset()

#
aanbevolenFilms = sc.parallelize(lijst_nieuw).groupByKey().mapValues(list)

#gesorteerdeFilms = aanbevolenFilms.groupByKey().mapValues(list)

#
joinFilmOpGebruiker = gebruikerFilms.join(aanbevolenFilms)

#
mapFilms = joinFilmOpGebruiker.map(lambda x: (x[0], x[1][0], x[1][1]))

# Laat tabel zien met gegevens
mapFilms.toDF(['Gebruiker ID', 'Bekeken Films', 'Aanbevolen films']).show()

print("Het laten zien van de bekeken en aanbevolen films duurde:", benchmark_get(), "\n")

+------------+--------------------+--------------------+
|Gebruiker ID|       Bekeken Films|    Aanbevolen films|
+------------+--------------------+--------------------+
|       35129|[1, 2, 7, 19, 39,...|[34, 17, 110, 454...|
|       54570|[110, 153, 165, 1...|[3793, 34, 17, 22...|
|      102075|[1, 3, 7, 11, 17,...|[110, 223, 1193, ...|
|       65378|[1, 3, 5, 7, 32, ...|[3793, 34, 17, 11...|
|      122346|[527, 788, 899, 9...|[3793, 34, 17, 11...|
|       47732|[27, 38, 318, 765...|[3793, 34, 17, 11...|
|       89829|[9, 17, 22, 61, 1...|[3793, 34, 110, 2...|
|      109769|[4, 34, 39, 41, 4...|[3793, 17, 110, 2...|
|        8231|[1, 3, 5, 6, 14, ...|[3793, 17, 110, 2...|
|       61988|[10, 296, 317, 50...|[3793, 34, 17, 11...|
|       36764|[1, 25, 32, 50, 6...|[3793, 34, 17, 11...|
|       99605|[50, 58, 123, 150...|[34, 17, 110, 589...|
|       69327|[1, 2, 6, 20, 23,...|[3793, 34, 17, 22...|
|       37728|[44, 198, 230, 26...|[3793, 34, 17, 11...|
|      103791|[2, 5, 10, 11, 16

In [7]:
# Aantal beoordelingen
testBeoordeling = csvRatingTest.map(lambda x: (x[0], x[1]))
testBeoordeling.count()

50000

In [8]:
benchmark_reset()

#
joinTabellen = joinFilmOpGebruiker.join(testBeoordeling)

# 
volledigeTabel = joinTabellen.map(lambda x: (x[0], x[1][0][0], x[1][0][1], x[1][1]))

# Sla alle gegevens op
result = volledigeTabel.toDF(['Gebuiker_ID', 'Bekeken_films', 'Aanbevolen_films', 'Volgende_Film'])

# Tel aantal gegevens erin
volledigeTabel.count()

# Laat tabel zien met gegevens
result.show()

print("Het joinen van de bekeken, aanbevolen en volgende film(s) duurde:", benchmark_get(), "\n")

+-----------+--------------------+--------------------+-------------+
|Gebuiker_ID|       Bekeken_films|    Aanbevolen_films|Volgende_Film|
+-----------+--------------------+--------------------+-------------+
|       7505|[1, 4, 5, 7, 11, ...|[3793, 1193, 589,...|         1455|
|      74770|[593, 1097, 1193,...|[3793, 34, 17, 11...|          296|
|       8231|[1, 3, 5, 6, 14, ...|[3793, 17, 110, 2...|          110|
|     111557|[19, 47, 110, 260...|[34, 17, 223, 589...|        60072|
|      15831|[2, 6, 10, 11, 12...|[3793, 34, 17, 22...|         1036|
|      25048|[912, 1127, 1210,...|[3793, 34, 17, 11...|         2688|
|     129913|[50, 64, 111, 356...|[3793, 34, 17, 11...|         5478|
|      73430|[47, 50, 110, 260...|[3793, 34, 17, 22...|         2730|
|     103199|[1, 2, 3, 5, 6, 1...|[4306, 34, 17, 11...|         3495|
|      98870|[43, 47, 50, 223,...|[34, 17, 110, 589...|          608|
|      70933|[1, 47, 110, 150,...|[3793, 34, 17, 22...|          736|
|     118154|[16, 32

In [11]:
result.registerTempTable("tempTabel")
# Aantal geraadde films
query = sqlContext.sql("""SELECT count(*) AS aantal_geraadde_films, (count(*)/50000)*100 AS Succes_Percentage FROM tempTabel WHERE array_contains(Aanbevolen_films, Volgende_Film) AND Gebuiker_ID = Gebuiker_ID""").show()




+---------------------+------------------+
|aantal_geraadde_films| Succes_Percentage|
+---------------------+------------------+
|                13370|26.740000000000002|
+---------------------+------------------+

